# **Chatbot memory**

**Introduction:**
The main focus is on handling prompts and maintaining context within the model's attention window.

**Objective:**
The aim is to address issues related to flattening conversation elements, catching exceptions when prompts exceed the model's attention window, and selecting appropriate truncation methods to preserve context.

**Procedure:**
1. **Flattening the Conversation:**
   - Flatten the list of conversational elements using delimiters, such as two new lines (e.g. `"".join()` or `custom flattener`).

2. **Handling Exceptions:**
   - Catch exceptions thrown by the LLM when prompts exceed its attention window.

3. **Truncation Methods:**
   - **Simple Truncation:** Remove the oldest element, which may result in losing important context, such as the system prompt.
   - **Selective Truncation:** Retain the system prompt while removing recent conversational turns, ensuring continuity of context.

**Important Points:**
- The choice of truncation method impacts the relevance and continuity of the LLM's responses. Retaining the system prompt can help maintain the LLM's identity and response consistency.

In [1]:
def installer():
    !pip install openai==0.28
    # !pip install --upgrade openai
installer()

In [2]:
import openai
import json
import os
import time

In [3]:
!pip show openai

Name: openai
Version: 0.28.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, requests, tqdm
Required-by: 


In [4]:
openai.api_key =  "voc-1126793339126677340548166ad2296cf93d9.28124368"
openai.api_base = "https://openai.vocareum.com/v1" # Remove this if using personal key

In [22]:
# Decoding parameters
TEMPERATURE = 0.0
MAX_TOKENS = 3950  # Increased to simulate LLM with smaller attention window
TOP_P = 1.0

In [23]:
SYSTEM_PROMPT = """You expert at games of chance.
End every response with double exclamation points!!"""

USER_NAME = "User"
AI_NAME = "AI Assistant"
NEW_INTERACTION_DELIMITER = "\n\n"

# Creating a chat bot with memory
Using the basic `openai.Completion` API to understand where the memory "lives" in the prompt.

In [27]:
def query_openai(prompt):

    # openai.Completion is no longer supported in openai>=1.0.0
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        top_p=TOP_P,
    )
    time.sleep(5) # to avoid rate limit errors
    if "error" in response:
        raise openai.InvalidRequestError(response["error"], param=None)
    else:
        return response.choices[0].text.strip().strip("\n")


def get_system_prompt(input_str=SYSTEM_PROMPT):
    return [f"System:{input_str}"]


def get_convo(input_str, convo):
    if not convo:
        convo = get_system_prompt()
    user_input_str = f"{USER_NAME}: {input_str}"
    response_trigger = f"{AI_NAME}: "
    convo.extend([user_input_str, response_trigger])
    return convo


def get_response(input_str, convo, use_simple_truncation, verbose):
    """
    Generate a response from an LLM based on user input_str and conversation history.

    Parameters:
    input_str (str): The user's current input_str or query to the language model.
    convo (list of str): A list representing the history of the conversation.
    use_simple_truncation (bool): A flag to determine whether to use a simple truncation
                                  method for managing conversation length.
    verbose (bool): A flag to determine if entire convo history should be printed.

    Returns:
    str: The generated response from the language model based on the current input_str and
         the conversation history.
    """
    convo = get_convo(input_str, convo)

    print(convo)

    # Try to prompt model and catch if the prompt exceeds the attention window
    first_try = True
    atten_window_all_used_up = False
    while first_try or atten_window_all_used_up:

        # Convo list flattened into string to feed to model - Method 1
        flattened_convo = ""
        def flattener(empty_str, strings_list):
            for string in strings_list:
                empty_str += string
            return empty_str
        flattened_convo = flattener(flattened_convo, convo)

        # Convo list flattened into string to feed to model - Method 2
        # flattened_convo = NEW_INTERACTION_DELIMITER.join(convo)

        try:
            first_try = False
            response = query_openai(flattened_convo)
            atten_window_all_used_up = False

        except openai.OpenAIError as e:
            atten_window_all_used_up = True
            if verbose:
                print("** ATTEN_WINDOW ALL USED UP **")
                print(f"OpenAI Error: {repr(e)}\n")

            if not convo:
                return [
                    "Our Error: System prompt is using up too many tokens of the attention window"
                ]

            # We can recover from over-allocation of atten_window by removing
            # components from history.
            if use_simple_truncation:
                # Just remove oldest element in convo
                convo = convo[1:]  # TODO

            else:
                # Remove the oldest User or AI convo turn, while retaining
                # system prompt
                convo = convo[:1] + convo[2:] # TODO

    # Add the LLM response to the response_trigger
    convo[-1] += response
    if verbose:
        print(NEW_INTERACTION_DELIMITER.join(convo))
    else:
        print(f"{USER_NAME}: {input_str}")
        print(f"{AI_NAME}: {response}")

    return convo


def chat(user_query, convo=[], use_simple_truncation=False, verbose=False):
    convo = get_response(user_query, convo, use_simple_truncation, verbose)
    return convo

## Testing our Chat bot

In [19]:
user_inputs = [
    "What cards game has the best odds of winning?",
    "What are the odds of winning it?",
    "What is the best hand to be dealt?",
    "What is the next most likely game to win?",
]

### Simple convo truncation method
Model remembers prior convo but **forgets to** always ends in exclamation points!!

In [28]:
convo = []
verbose = True
simple_truncation = True
for i, input in enumerate(user_inputs):
    print(f"**** Convo turn {i} ****")
    convo = chat(
        input, convo=convo, use_simple_truncation=simple_truncation, verbose=verbose
    )
    print()

**** Convo turn 0 ****
['System:You expert at games of chance.\nEnd every response with double exclamation points!!', 'User: What cards game has the best odds of winning?', 'AI Assistant: ']
System:You expert at games of chance.
End every response with double exclamation points!!

User: What cards game has the best odds of winning?

AI Assistant: Blackjack is known for having some of the best odds in a casino!!

**** Convo turn 1 ****
['System:You expert at games of chance.\nEnd every response with double exclamation points!!', 'User: What cards game has the best odds of winning?', 'AI Assistant: Blackjack is known for having some of the best odds in a casino!!', 'User: What are the odds of winning it?', 'AI Assistant: ']
System:You expert at games of chance.
End every response with double exclamation points!!

User: What cards game has the best odds of winning?

AI Assistant: Blackjack is known for having some of the best odds in a casino!!

User: What are the odds of winning it?

AI 

#### What is happening?
Peaking under hood to see the full conversation as it grows

NOTE: In **convo turn 3** we remove the system prompt to avoid exceeding the LLM attention window


In [ ]:
convo = []
verbose = True
simple_truncation = True
for i, input in enumerate(user_inputs):
    print(f"**** Convo turn {i} ****")
    convo = chat(
        input, convo=convo, use_simple_truncation=simple_truncation, verbose=verbose
    )
    print()

**** Convo turn 0 ****
System:You expert at games of chance.
End every response with double exclamation points!!

User: What cards game has the best odds of winning?

AI Assistant: The game of blackjack has the best odds of winning, with a house edge of only 1%!

**** Convo turn 1 ****
System:You expert at games of chance.
End every response with double exclamation points!!

User: What cards game has the best odds of winning?

AI Assistant: The game of blackjack has the best odds of winning, with a house edge of only 1%!

User: What are the odds of winning it?

AI Assistant: The odds of winning at blackjack depend on various factors, such as the number of decks in play and the specific rules of the game. However, with proper strategy and a bit of luck, you have a good chance of coming out on top!!

**** Convo turn 2 ****
System:You expert at games of chance.
End every response with double exclamation points!!

User: What cards game has the best odds of winning?

AI Assistant: The game 

### Retaining System prompt truncation method
Model remembers most of prior convo and **remembers to** always ends in exclamation points!!

In [ ]:
convo = []
verbose = False
for i, input in enumerate(user_inputs):
    print(f"**** Convo turn {i} ****")
    convo = chat(input, convo=convo, verbose=verbose)
    print()

**** Convo turn 0 ****
User: What cards game has the best odds of winning?
AI Assistant: The game of blackjack has the best odds of winning, with a house edge of only 1%!

**** Convo turn 1 ****
User: What are the odds of winning it?
AI Assistant: The odds of winning at blackjack depend on various factors, such as the number of decks in play and the specific rules of the game. However, with proper strategy and a bit of luck, you have a good chance of coming out on top!!

**** Convo turn 2 ****
User: What is the best hand to be dealt?
AI Assistant: The best hand to be dealt in blackjack is a natural blackjack, which is an ace and a 10-value card. This hand pays out at 3:2 and gives you an immediate advantage over the dealer!!

**** Convo turn 3 ****
User: What is the next most likely game to win?
AI Assistant: The next most likely game to win would depend on your personal preferences and skills. Some popular games of chance include poker, roulette, and baccarat. However, with your exper

#### What is happening?
Peaking under hood to see full convo as it grows

NOTE: In **convo turn 3** we remove the oldest **convo turns**, but retain the system prompt

In [ ]:
convo = []
verbose = True
for i, input in enumerate(user_inputs):
    print(f"**** Convo turn {i} ****")
    convo = chat(input, convo=convo, verbose=verbose)
    print()

**** Convo turn 0 ****
System:You expert at games of chance.
End every response with double exclamation points!!

User: What cards game has the best odds of winning?

AI Assistant: The game of blackjack has the best odds of winning, with a house edge of only 1%!

**** Convo turn 1 ****
System:You expert at games of chance.
End every response with double exclamation points!!

User: What cards game has the best odds of winning?

AI Assistant: The game of blackjack has the best odds of winning, with a house edge of only 1%!

User: What are the odds of winning it?

AI Assistant: The odds of winning at blackjack depend on various factors, such as the number of decks in play and the specific rules of the game. However, with proper strategy and a bit of luck, you have a good chance of coming out on top!!

**** Convo turn 2 ****
System:You expert at games of chance.
End every response with double exclamation points!!

User: What cards game has the best odds of winning?

AI Assistant: The game 